In [9]:
import pandas as pd
import numpy as np

In [10]:
def filter_data(dimension, step, cpu, gpu, mem, cores, bs, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals):


    if step == 'next':
        if dimension == 'cores':
            # Get index of cores from core_vals
            core_index = core_vals.index(cores)
            last_index = len(core_vals) - 1
            new_index = int(np.ceil((core_index+last_index)/2))
            value = core_vals[new_index]
            core_vals = core_vals[core_index+1:]

        elif dimension == 'cpu':
            # Get index of cpu from cpu_vals
            cpu_index = cpu_vals.index(cpu)
            last_index = len(cpu_vals) - 1
            new_index = int(np.ceil((cpu_index+last_index)/2))
            value = cpu_vals[new_index]
            cpu_vals = cpu_vals[cpu_index+1:]

        elif dimension == 'gpu':
            # Get index of gpu from gpu_vals
            gpu_index = gpu_vals.index(gpu)
            last_index = len(gpu_vals) - 1
            new_index = int(np.ceil((gpu_index+last_index)/2))
            value = gpu_vals[new_index]
            gpu_vals = gpu_vals[gpu_index+1:]

        elif dimension == 'mem':
            # Get index of mem from mem_vals
            mem_index = mem_vals.index(mem)
            last_index = len(mem_vals) - 1
            new_index = int(np.ceil((mem_index+last_index)/2))
            value = mem_vals[new_index]
            mem_vals = mem_vals[mem_index+1:]

        else:
            if len(bs_vals) == 2:
                value = bs_vals[1]
                bs_vals = bs_vals[1:]
            # Get index of bs from bs_vals
            bs_index = bs_vals.index(bs)
            last_index = len(bs_vals) - 1
            new_index = int(np.ceil((bs_index+last_index)/2))
            value = bs_vals[new_index]
            bs_vals = bs_vals[bs_index+1:]

    else:

        if dimension == 'cores':
            # Get index of cores from core_vals
            core_index = core_vals.index(cores)
            new_index = int(np.floor(core_index/2))
            value = core_vals[new_index]
            core_vals = core_vals[:core_index]
        
        elif dimension == 'cpu':
            # Get index of cpu from cpu_vals
            cpu_index = cpu_vals.index(cpu)
            new_index = int(np.floor(cpu_index/2))
            value = cpu_vals[new_index]
            cpu_vals = cpu_vals[:cpu_index]

        elif dimension == 'gpu':
            # Get index of gpu from gpu_vals
            gpu_index = gpu_vals.index(gpu)
            new_index = int(np.floor(gpu_index/2))
            value = gpu_vals[new_index]
            gpu_vals = gpu_vals[:gpu_index]

        elif dimension == 'mem':
            # Get index of mem from mem_vals
            mem_index = mem_vals.index(mem)
            new_index = int(np.floor(mem_index/2))
            value = mem_vals[new_index]
            mem_vals = mem_vals[:mem_index]

            # mem_index = mem_vals.index(mem)
            # new_index = int(np.floor(len(mem_vals)/2))
            # value = mem_index
            # mem_vals = mem_vals[:mem_index+1]

        else:
            # Get index of bs from bs_vals
            bs_index = bs_vals.index(bs)
            new_index = int(np.floor(bs_index/2))
            value = bs_vals[new_index]
            bs_vals = bs_vals[:bs_index]

    return value, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals


In [11]:
def optimize(data, power_budget, runs):

    index_list = []

    core_vals=[4, 8, 12] #3 possible values
    gpu_vals=[114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
    cpu_vals=[422400, 729600, 1036800, 1344000, 1651200, 1958400, 2201600] #in kHz, 7 possible values
    mem_vals = [665600000, 2133000000, 3199000000]
    bs_vals = [1]

    best_powermode = str(8) + '_' + str(1344000) + '_' + str(726750000) + '_' + str(2133000000)
    best_power = 0
    best_time = 99999

    core_mid = 8
    cpu_mid = 1344000
    gpu_mid = 726750000
    mem_mid = 2133000000 
    bs_mid = 1

    curr_core = core_mid
    curr_cpu = cpu_mid
    curr_gpu = gpu_mid
    curr_mem = mem_mid
    curr_bs = bs_mid

    mid_index = data[(data['cores'] == core_mid) & (data['cpu'] == cpu_mid) & (data['gpu'] == gpu_mid) & (data['mem'] == mem_mid) & (data['bs'] == bs_mid)].index[0]
    curr_index = mid_index
    curr_index_power = data.loc[mid_index, 'observed_power']
    curr_index_time = data.loc[mid_index, 'observed_time']

    order = ['cpu', 'gpu', 'cores', 'mem', 'bs']
       # curr_order = [curr_gpu, curr_cpu, curr_core, curr_mem]
    start_powermode = str(curr_core) + '_' + str(curr_cpu) + '_' + str(curr_gpu) + '_' + str(curr_mem)
    print("Start powermode: ",start_powermode)
    space = len(core_vals) * len(cpu_vals) * len(gpu_vals) * len(mem_vals) * len(bs_vals)
    print("Space left: ", space)
    for i in range(runs):
        count = 0
        for i in order:
            
            if curr_index_power <= power_budget:

                if i == 'gpu':
                    curr_gpu, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals  = filter_data(i,'next', curr_cpu, curr_gpu, curr_mem, curr_core, curr_bs, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals)
                elif i == 'cpu':
                    curr_cpu, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals = filter_data(i,'next', curr_cpu, curr_gpu, curr_mem, curr_core, curr_bs, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals)
                elif i == 'mem':
                    curr_mem, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals = filter_data(i, 'next',curr_cpu, curr_gpu, curr_mem, curr_core, curr_bs, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals)
                elif i == 'cores':
                    curr_core, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals = filter_data(i,'next', curr_cpu, curr_gpu, curr_mem, curr_core, curr_bs, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals)
                else:
                    curr_bs, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals = filter_data(i,'next', curr_cpu, curr_gpu, curr_mem, curr_core, curr_bs, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals)
                curr_index = data[(data['cores'] == curr_core) & (data['cpu'] == curr_cpu) & (data['gpu'] == curr_gpu) & (data['mem'] == curr_mem) & (data['bs'] == curr_bs)].index[0]
                curr_index_power = data.loc[curr_index, 'observed_power']
                curr_index_time = data.loc[curr_index, 'observed_time']
                index_list.append(curr_index)
                powermode = str(curr_core) + '_' + str(curr_cpu) + '_' + str(curr_gpu) + '_' + str(curr_mem) + '_' + str(curr_bs)
                print("Current Powermode: ", powermode)
                print("Current Power: ", curr_index_power)
                print("Current Time: ", curr_index_time)

                print("Current Core Vals: ", core_vals)
                print("Current CPU Vals: ", cpu_vals)
                print("Current GPU Vals: ", gpu_vals)
                print("Current Mem Vals: ", mem_vals)
                print("Current BS Vals: ", bs_vals)

                space = len(core_vals) * len(cpu_vals) * len(gpu_vals) * len(mem_vals) * len(bs_vals)
                print("Space left: ", space)
                
                if space == 0:
                    break

            else:
                if i == 'gpu':
                    curr_gpu, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals  = filter_data(i,'prev', curr_cpu, curr_gpu, curr_mem, curr_core, curr_bs, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals)
                elif i == 'cpu':
                    curr_cpu, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals = filter_data(i,'prev', curr_cpu, curr_gpu, curr_mem, curr_core, curr_bs, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals)
                elif i == 'mem':
                    curr_mem, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals = filter_data(i, 'prev',curr_cpu, curr_gpu, curr_mem, curr_core, curr_bs, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals)
                elif i == 'cores':
                    curr_core, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals = filter_data(i,'prev', curr_cpu, curr_gpu, curr_mem, curr_core, curr_bs, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals)
                else:
                    curr_bs, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals = filter_data(i,'prev', curr_cpu, curr_gpu, curr_mem, curr_core, curr_bs, core_vals, cpu_vals, gpu_vals, mem_vals, bs_vals)

 
                curr_index = data[(data['cores'] == curr_core) & (data['cpu'] == curr_cpu) & (data['gpu'] == curr_gpu) & (data['mem'] == curr_mem) & (data['bs'] == curr_bs)].index[0]
                curr_index_power = data.loc[curr_index, 'observed_power']
                curr_index_time = data.loc[curr_index, 'observed_time']
                index_list.append(curr_index)
                powermode = str(curr_core) + '_' + str(curr_cpu) + '_' + str(curr_gpu) + '_' + str(curr_mem) + '_' + str(curr_bs)
                print("Current Powermode: ", powermode)
                print("Current Power: ", curr_index_power)
                print("Current Time: ", curr_index_time)

                print("Current Core Vals: ", core_vals)
                print("Current CPU Vals: ", cpu_vals)
                print("Current GPU Vals: ", gpu_vals)
                print("Current Mem Vals: ", mem_vals)

                space = len(core_vals) * len(cpu_vals) * len(gpu_vals) * len(mem_vals)
                print("Space left: ", space)

                if space == 0:
                    break  
                    

            count+=1

    return index_list 
        

In [12]:
model = 'resnet'
data = pd.read_csv(f'{model}_infer_data_final.csv')

In [13]:
core_vals=[4, 8, 12] #3 possible values
gpu_vals=[114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
cpu_vals=[422400, 729600, 1036800, 1344000, 1651200, 1958400, 2201600] #in kHz, 7 possible values
mem_vals = [665600000, 2133000000, 3199000000]
bs_vals = [1]

# Filter data based on values in the list only
data = data[data['cores'].isin(core_vals)]
data = data[data['gpu'].isin(gpu_vals)]
data = data[data['cpu'].isin(cpu_vals)]
data = data[data['mem'].isin(mem_vals)]
data = data[data['bs'].isin(bs_vals)]

In [14]:
data

,cores,cpu,gpu,mem,observed_time,bs,observed_power
0,4,422400,114750000,665600000,77.866302,1.0,13.0460
1,4,422400,114750000,2133000000,80.801651,1.0,13.7975
2,4,422400,114750000,3199000000,71.439362,1.0,14.8525
3,8,422400,114750000,665600000,71.634914,1.0,13.4470
4,8,422400,114750000,2133000000,68.684834,1.0,14.4490
...,...,...,...,...,...,...,...
436,8,2201600,1300500000,2133000000,17.407376,1.0,25.0740
437,8,2201600,1300500000,3199000000,16.200047,1.0,25.5230
438,12,2201600,1300500000,665600000,21.749041,1.0,22.4635
439,12,2201600,1300500000,2133000000,16.482096,1.0,23.8685


In [15]:
optimize(data, 31, 2)

Start powermode:  8_1344000_726750000_2133000000
Space left:  441
Current Powermode:  8_1958400_726750000_2133000000_1
Current Power:  19.464
Current Time:  19.654128074646
Current Core Vals:  [4, 8, 12]
Current CPU Vals:  [1651200, 1958400, 2201600]
Current GPU Vals:  [114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
Current Mem Vals:  [665600000, 2133000000, 3199000000]
Current BS Vals:  [1]
Space left:  189
Current Powermode:  8_1958400_1134750000_2133000000_1
Current Power:  21.7155
Current Time:  19.269248008728027
Current Core Vals:  [4, 8, 12]
Current CPU Vals:  [1651200, 1958400, 2201600]
Current GPU Vals:  [930750000, 1134750000, 1300500000]
Current Mem Vals:  [665600000, 2133000000, 3199000000]
Current BS Vals:  [1]
Space left:  81
Current Powermode:  12_1958400_1134750000_2133000000_1
Current Power:  21.915
Current Time:  17.799296379089355
Current Core Vals:  [12]
Current CPU Vals:  [1651200, 1958400, 2201600]
Current GPU Vals:  [930750000, 113

[346, 364, 367, 368, 368, 431]

In [16]:
# Power list rom 10 to 50 in increments of 1
if model =='bert':
    power_list = list(range(10, 61))
else:
    power_list = list(range(10, 51))

In [17]:
def pick_best_powermode(index_list, data, power_budget, time_budget, a):
    # filter data based on the index_list
    filtered_data = data.iloc[index_list]
    # filter data based on the power budget
    filtered_data = filtered_data[filtered_data['observed_power'] <= power_budget]
    filtered_data['time_scaled'] = filtered_data['observed_time']/1000.0
    filtered_data['time_cond1'] = (filtered_data['bs'].astype(int))/a + filtered_data['time_scaled']
    filtered_data['time_cond2'] = filtered_data['time_scaled']*a

    filtered_data = filtered_data[(filtered_data['time_cond1'] <= time_budget)]
    filtered_data = filtered_data[(filtered_data['time_cond2'] <= filtered_data['bs'])]
    # sort the data based on the observed time
    filtered_data = filtered_data.sort_values(by='time_scaled')
    # pick the first sample
    best_sample = filtered_data.iloc[0]
    return best_sample

In [18]:
time_budget = 0.1
arr_rate = 30

In [19]:
df_1 = pd.DataFrame(columns=['cpu','gpu','mem','cores', 'bs','observed_time','observed_power','power_budget','num_pwds'])


for i in power_list:
    try:
        index_list = optimize(data, i, 2)   
        powermode_index = pick_best_powermode(index_list, data, i, time_budget, arr_rate)
        cpu = powermode_index['cpu']
        gpu = powermode_index['gpu']
        mem = powermode_index['mem']
        cores = powermode_index['cores']
        bs = powermode_index['bs']
        time = powermode_index['observed_time']
        power = powermode_index['observed_power']
        df_1.loc[len(df_1)] = [cpu, gpu, mem, cores,  bs, time, power, i, 7]

    except:
        print("No solution for power budget: ", i)    
        df_1.loc[len(df_1)] = [0, 0, 0, 0, 0, 0, 0, i, 7]
    

Start powermode:  8_1344000_726750000_2133000000
Space left:  441
Current Powermode:  8_729600_726750000_2133000000_1
Current Power:  15.652
Current Time:  40.96198272705078
Current Core Vals:  [4, 8, 12]
Current CPU Vals:  [422400, 729600, 1036800]
Current GPU Vals:  [114750000, 318750000, 522750000, 726750000, 930750000, 1134750000, 1300500000]
Current Mem Vals:  [665600000, 2133000000, 3199000000]
Space left:  189
Current Powermode:  8_729600_318750000_2133000000_1
Current Power:  15.048
Current Time:  41.67803192138672
Current Core Vals:  [4, 8, 12]
Current CPU Vals:  [422400, 729600, 1036800]
Current GPU Vals:  [114750000, 318750000, 522750000]
Current Mem Vals:  [665600000, 2133000000, 3199000000]
Space left:  81
Current Powermode:  4_729600_318750000_2133000000_1
Current Power:  15.055
Current Time:  42.495920181274414
Current Core Vals:  [4]
Current CPU Vals:  [422400, 729600, 1036800]
Current GPU Vals:  [114750000, 318750000, 522750000]
Current Mem Vals:  [665600000, 213300000

In [20]:
def find_optimal_pm(df, pmax, tmax, a):
    #filter rows where interleaved_power<=pmax, sort in ascending order of interleaved_time
    df = df[df['observed_power'] <= pmax]
    df['time_scaled'] = df['observed_time']/1000.0
    df['time_cond1'] = (df['bs'].astype(int))/a + df['time_scaled']
    df['time_cond2'] = df['time_scaled']*a

    df = df[(df['time_cond1'] <= tmax)]
    df = df[(df['time_cond2'] <= df['bs'])]
    
    df = df.sort_values(by='time_scaled', ascending=True)
    #return top1 powermode, time and power
    return df.iloc[0]['cores'], df.iloc[0]['cpu'], df.iloc[0]['gpu'], df.iloc[0]['mem'], df.iloc[0]['bs'], df.iloc[0]['observed_time'], df.iloc[0]['observed_power']

In [21]:
optimal_df = pd.DataFrame(columns=['power_budget', 'cores', 'cpu', 'gpu', 'mem', 'bs', 'optim_time', 'optim_power'])
for power_budget in power_list:
    try:
        cores, cpu, gpu, mem, bs, algo_time, algo_power = find_optimal_pm(data, power_budget, time_budget, arr_rate)
        optimal_df.loc[len(optimal_df)] = [power_budget, cores, cpu, gpu, mem, bs, algo_time, algo_power]
    except:
        print("No Power Mode Found")
        optimal_df.loc[len(optimal_df)] = [power_budget, 0, 0, 0, 0, 0, 0, 0]

No Power Mode Found
No Power Mode Found
No Power Mode Found
No Power Mode Found
No Power Mode Found
No Power Mode Found


/tmp/ipykernel_2666294/3052255314.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_scaled'] = df['observed_time']/1000.0
/tmp/ipykernel_2666294/3052255314.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_cond1'] = (df['bs'].astype(int))/a + df['time_scaled']
/tmp/ipykernel_2666294/3052255314.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [22]:
merged_df = pd.merge(df_1, optimal_df, on='power_budget', suffixes=('_bs', '_optim'))

In [23]:
merged_df

,cpu_bs,gpu_bs,mem_bs,cores_bs,bs_bs,observed_time,observed_power,power_budget,num_pwds,cores_optim,cpu_optim,gpu_optim,mem_optim,bs_optim,optim_time,optim_power
0,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.0000,10.0,7.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.0000
1,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.0000,11.0,7.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.0000
2,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.0000,12.0,7.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.0000
3,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.0000,13.0,7.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.0000
4,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.0000,14.0,7.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.0000
5,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.0000,15.0,7.0,0.0,0.0,0.000000e+00,0.000000e+00,0.0,0.000000,0.0000
6,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.0000,16.0,7.0,4.0,1651200.0,3.187500e+08,6.656000e+08,1.0,27.364112,15.7560
7,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.000000,0.0000,17.0,7.0,12.0,1651200.0,5.227500e+08,6.656000e+08,1.0,25.029840,16.9580
8,1958400.0,3.187500e+08,2.133000e+09,8.0,1.0,21.163744,17.9180,18.0,7.0,8.0,1958400.0,3.187500e+08,2.133000e+09,1.0,21.163744,17.9180
9,1958400.0,3.187500e+08,2.133000e+09,8.0,1.0,21.163744,17.9180,19.0,7.0,8.0,1958400.0,5.227500e+08,2.133000e+09,1.0,19.047296,18.2625


In [ ]:
time = int(time_budget*1000)

In [24]:
merged_df.to_csv(f'{model}_infer_only_bs_vs_optim_bs1.csv', index=False)